In [3]:
import torch
torch.__version__
torch.cuda.is_available()

True

In [5]:
def read_new_email(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    parts = text.split('"')  
    
    subject = parts[1].strip()
    body = parts[3].strip()
    label = parts[5].strip()
    
    new_email = {'Subject':subject, 'Body':body, 'Label':label}
    return new_email

{'Subject': 'your application to the position of 2024 it intern – 31060216',
 'Body': 'dear [name] ,we received your application for the position of 2024 it intern – 31060216 and are reviewing your experience and qualifications. if you meet the requirements of the position, a member of our talent acquisition team will contact you. you can view your application status by signing in to your candidate home using the same email and password used when you applied to the job. once signed in, click on your email address in upper right and select view profile. under my applications is where you can view the status of the jobs you have applied to. thank you for your interest in abbott. please feel free to learn more about our company, and continue to view open positions, by visiting our website at www.jobs.abbott.com. to keep up to date with abbott news, click the links and connect with us:',
 'Label': 'confirmation'}

In [7]:
confirmation_emails = []

for i in range(1, 92):
    file_path = f"Data/confirmation/conf{i}.txt"
    new_email = read_new_email(file_path)
    confirmation_emails.append(new_email)

for i in range(0,10):
    print(f"Email #{i+1}: {confirmation_emails[i].get('Label')}")

Email #1: confirmation
Email #2: confirmation
Email #3: confirmation
Email #4: confirmation
Email #5: confirmation
Email #6: confirmation
Email #7: confirmation
Email #8: confirmation
Email #9: confirmation
Email #10: confirmation
